<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_p3_UNetplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [3]:
import os


train_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot'
img_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.jpg')])

In [4]:
from PIL import Image


class LandDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        return img, os.path.basename(self.img_paths[idx])

In [5]:
from torchvision import transforms as tr

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

test_ds = LandDataset(img_paths_train,
                       transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                      )

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
from torchvision.models.segmentation.deeplabv3 import deeplabv3_resnet101, DeepLabHead, FCNHead
from torch import nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
        super(ConvBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def __call__(self, x):
        return self.convblock(x)

class UNet(nn.Module):
    def __init__(self, num_classes=7, in_channels=3, init_channels=16):
        super(UNet, self).__init__()
        # Encoder
        self.encoder1 = ConvBlock(in_channels, init_channels) # (3, H, W) -> (64, H, W)
        self.pooling1 = nn.MaxPool2d(2)
        self.encoder2 = ConvBlock(init_channels, init_channels*2) # (64, H/2, W/2) -> (128, H/2, W/2)
        self.pooling2 = nn.MaxPool2d(2)
        self.encoder3 = ConvBlock(init_channels*2, init_channels*4) # (128, H/4, W/4) -> (256, H/4, W/4)
        self.pooling3 = nn.MaxPool2d(2)
        self.encoder4 = ConvBlock(init_channels*4, init_channels*8) # (256, H/8, W/8) -> (512, H/8, W/8)
        self.pooling4 = nn.MaxPool2d(2)
        self.encoder5 = ConvBlock(init_channels*8, init_channels*16) # (256, H/8, W/8) -> (512, H/8, W/8)
        self.pooling5 = nn.MaxPool2d(2)
        self.encoder6 = ConvBlock(init_channels*16, init_channels*32) # (512, H/16, W/16) -> (1024, H/16, W/16)

        # Decoder
        self.upconv5 = nn.ConvTranspose2d(init_channels*32, init_channels*16, kernel_size=2, stride=2) # (1024, H/16, W/16) -> (512, H/8, W/8)
        self.decoder5 = ConvBlock(init_channels*32, init_channels*16)
        self.upconv4 = nn.ConvTranspose2d(init_channels*16, init_channels*8, kernel_size=2, stride=2) # (1024, H/16, W/16) -> (512, H/8, W/8)
        self.decoder4 = ConvBlock(init_channels*16, init_channels*8)
        self.upconv3 = nn.ConvTranspose2d(init_channels*8, init_channels*4, kernel_size=2, stride=2) # (512, H/8, W/8) -> (256, H/4, W/4)
        self.decoder3 = ConvBlock(init_channels*8, init_channels*4)
        self.upconv2 = nn.ConvTranspose2d(init_channels*4, init_channels*2, kernel_size=2, stride=2) # (256, H/4, W/4) -> (128, H/2, W/2)
        self.decoder2 = ConvBlock(init_channels*4, init_channels*2)
        self.upconv1 = nn.ConvTranspose2d(init_channels*2, init_channels, kernel_size=2, stride=2) # (128, H/2, W/2) -> (64, H, W)
        self.decoder1 = ConvBlock(init_channels*2, init_channels)

        # Output
        self.output = nn.Conv2d(init_channels, out_channels=num_classes, kernel_size=1) # (64, H, W) -> (num_classes, H, W)

    def __call__(self, x):
        # Encoder
        encode1 = self.encoder1(x)
        encode2 = self.encoder2(self.pooling1(encode1))
        encode3 = self.encoder3(self.pooling2(encode2))
        encode4 = self.encoder4(self.pooling3(encode3))
        encode5 = self.encoder5(self.pooling4(encode4))
        bottleneck = self.encoder6(self.pooling5(encode5))

        # Decoder
        x = torch.cat((self.upconv5(bottleneck), encode5), dim=1) # (1024, H/8, W/8)
        x = self.decoder5(x) # (512, H/8, W/8)
        x = torch.cat((self.upconv4(x), encode4), dim=1) # (1024, H/8, W/8)
        x = self.decoder4(x) # (512, H/8, W/8)
        x = torch.cat((self.upconv3(x), encode3), dim=1) # (512, H/4, W/4)
        x = self.decoder3(x) # (256, H/4, W/4)
        x = torch.cat((self.upconv2(x), encode2), dim=1) # (512, H/2, W/2)
        x = self.decoder2(x) # (128, H/4, W/4)
        x = torch.cat((self.upconv1(x), encode1), dim=1) # (128, H, W)
        x = self.decoder1(x) # (64, H, W)
        x = self.output(x)

        return x

def get_model(path):
    model = UNet()
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    model.eval()
    return model

In [7]:
import numpy as np
import imageio


def predimg(preds, names, out_path):
    for pred, name in zip(preds, names):
        pred = pred.detach().cpu().numpy()
        pred_img = np.zeros((512, 512, 3), dtype=np.uint8)
        pred_img[np.where(pred == 0)] = [0, 255, 255]
        pred_img[np.where(pred == 1)] = [255, 255, 0]
        pred_img[np.where(pred == 2)] = [255, 0, 255]
        pred_img[np.where(pred == 3)] = [0, 255, 0]
        pred_img[np.where(pred == 4)] = [0, 0, 255]
        pred_img[np.where(pred == 5)] = [255, 255, 255]
        pred_img[np.where(pred == 6)] = [0, 0, 0]
        new_name = name.replace('sat', 'mask').replace('.jpg', '.png')
        imageio.imwrite(os.path.join(out_path, new_name), pred_img)

In [9]:
epoch_1 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/epoch1model.pth')
epoch_40 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/epoch40model.pth')
epoch_80 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/BestIOU.pth')

output_directorys = ['/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/first/',
                    '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/mid/',
                    '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/last/']

In [14]:
for model, output_directory in zip([epoch_1, epoch_40, epoch_80], output_directorys):
    try:
        os.makedirs(output_directory, exist_ok=True)
    except:
        pass
    for x, filenames in test_loader:
        with torch.no_grad():
            x = x.to(device)
            pred_out = model(x)
        pred_out = pred_out.argmax(dim=1)
        predimg(pred_out, filenames, output_directory)